In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_path = '/content/drive/MyDrive/DL_PROJ/DLProject/train_labels.csv'
test_path = '/content/drive/MyDrive/DL_PROJ/DLProject/test_labels.csv'
images_path = '/content/drive/MyDrive/DL_PROJ/DLProject/DATASET'

In [ ]:
import pandas as pd
import numpy as np
import cv2 as cv
import os
import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam

import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

import plotly.graph_objects as go

In [ ]:
train_csv = pd.read_csv(train_path)
test_csv = pd.read_csv(test_path)

In [ ]:
import cv2

# Load the image
img = cv2.imread(images_path + '/test/1/test_0002_aligned.jpg')

# Get dimensions
height, width, channels = img.shape
print(f'Dimensions: {width}x{height}')

Dimensions: 100x100


In [ ]:
train_label_counts = train_csv['label'].value_counts()
test_label_counts =  test_csv['label'].value_counts()

In [ ]:
import plotly.express as px

# Plot bar graph
fig = px.bar(train_label_counts, x=train_label_counts.index, y=train_label_counts.values, color=train_label_counts.index, labels={'x': 'Labels', 'y': 'Count'})
fig.show()

In [ ]:
import plotly.express as px

# Plot bar graph
fig = px.bar(test_label_counts, x=test_label_counts.index, y=test_label_counts.values, color=test_label_counts.index, labels={'x': 'Labels', 'y': 'Count'})
fig.show()

In [ ]:
# Contains the RAFDB Training data
# Create a data generator with augmentation
train_data_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
)

rafdb_training_data = train_data_generator.flow_from_directory(
    images_path +'/train',
    target_size = (100, 100),
    batch_size = 64,
    color_mode = 'grayscale',
    class_mode = 'categorical'
)
rafdb_training_data

Found 12271 images belonging to 7 classes.


In [ ]:
# Contains the rafdb Test data
test_data_generator = ImageDataGenerator(rescale=1./255)
rafdb_test_data = test_data_generator.flow_from_directory(
    images_path + '/test',
    target_size = (100, 100),
    batch_size = 64,
    color_mode = 'grayscale',
    class_mode = 'categorical'
)
rafdb_test_data

Found 3068 images belonging to 7 classes.


In [ ]:
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [ ]:
from tensorflow.keras import optimizers
optims = [
    optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name='Nadam'),
    optimizers.Adam(0.001),
]

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()

# Convolutional Layers
model.add(Conv2D(filters=128, kernel_size=(3,3), input_shape=(100, 100, 1), activation='elu', padding='same', kernel_initializer='he_normal'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='elu', padding='same', kernel_initializer='he_normal'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='elu', padding='same', kernel_initializer='he_normal'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Dense Layers
model.add(Dense(128, activation='elu', kernel_initializer='he_normal'))
model.add(Dropout(0.25))

model.add(Dense(64, activation='elu', kernel_initializer='he_normal'))
model.add(Dropout(0.25))

# Output Layer
model.add(Dense(7, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 100, 100, 128)     1280      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 50, 50, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 50, 50, 64)        73792     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 25, 25, 64)        0         
 g2D)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 25, 25, 64)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 25, 25, 64)       

In [ ]:
# Reduce learning rate when a metric has stopped improving
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00005,
    patience=11,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=7,
    min_lr=1e-7,
    verbose=1,
)

callbacks = [
    early_stopping,
    lr_scheduler,
]

In [ ]:
batch_size = 64
history = model.fit(
    rafdb_training_data,
    epochs=60,
    validation_data=rafdb_test_data,
    batch_size = 64,
    callbacks=callbacks,
)

Epoch 1/60
192/192 [==============================] - 43s 212ms/step - loss: 2.0035 - accuracy: 0.2980 - val_loss: 9.0561 - val_accuracy: 0.0235 - lr: 0.0010
Epoch 2/60
192/192 [==============================] - 41s 212ms/step - loss: 1.7585 - accuracy: 0.3335 - val_loss: 9.7735 - val_accuracy: 0.0241 - lr: 0.0010
Epoch 3/60
192/192 [==============================] - 41s 215ms/step - loss: 1.7104 - accuracy: 0.3556 - val_loss: 1.6115 - val_accuracy: 0.3902 - lr: 0.0010
Epoch 4/60
192/192 [==============================] - 44s 227ms/step - loss: 1.6939 - accuracy: 0.3622 - val_loss: 1.6504 - val_accuracy: 0.3869 - lr: 0.0010
Epoch 5/60
192/192 [==============================] - 41s 213ms/step - loss: 1.6575 - accuracy: 0.3797 - val_loss: 1.5751 - val_accuracy: 0.3924 - lr: 0.0010
Epoch 6/60
192/192 [==============================] - 41s 211ms/step - loss: 1.6405 - accuracy: 0.3832 - val_loss: 1.5552 - val_accuracy: 0.4214 - lr: 0.0010
Epoch 7/60
192/192 [==============================] 

In [ ]:
# Create traces
fig = go.Figure()

# Training vs. Validation Accuracy
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['accuracy']) + 1)), y=history.history['accuracy'], mode='lines+markers', name='Training Accuracy'))
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['val_accuracy']) + 1)), y=history.history['val_accuracy'], mode='lines+markers', name='Validation Accuracy'))

# Layout for Accuracy
fig.update_layout(title='Training vs. Validation Accuracy', xaxis_title='Epoch', yaxis_title='Accuracy', template="plotly_white")

# Show the plot
fig.show()

# New figure for loss
fig = go.Figure()

# Training vs. Validation Loss
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['loss']) + 1)), y=history.history['loss'], mode='lines+markers', name='Training Loss'))
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['val_loss']) + 1)), y=history.history['val_loss'], mode='lines+markers', name='Validation Loss'))

# Layout for Loss
fig.update_layout(title='Training vs. Validation Loss', xaxis_title='Epoch', yaxis_title='Loss', template="plotly_white")

# Show the plot
fig.show()

In [ ]:
# Evaluate the model on the validation dataset
loss, accuracy = model.evaluate(rafdb_test_data, verbose=1)

# Print the loss and accuracy
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

48/48 [==============================] - 5s 105ms/step - loss: 0.7425 - accuracy: 0.7500
Validation Loss: 0.7424852252006531
Validation Accuracy: 75.00%
